# 1.2 EntropyMeasure - Randomized Measurement
## Multiple Experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import EntropyMeasure

experiment_randomized = EntropyMeasure()
# It's default method. EntropyMeasure(method='randomized') also works

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_randomized.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_randomized.add(GHZ(i), f"ghz_{i}")
    experiment_randomized.add(make_neel_circuit(i), f"neel_{i}")

experiment_randomized.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753a35eb7320>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x753a35eb73e0>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753a364ef6e0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753a35eb6780>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x753a8197e330>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753a35e30320>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753a35eec980>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x753a35eeca10>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x753a35eecc50>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x753a35eece00>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x753a35eece90>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.randomized_measure.arguments import EntropyMeasureRandomizedMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class EntropyMeasureRandomizedMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function :func:`generate_random_unitary_seeds` 
    in :mod:`qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds

        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[EntropyMeasureRandomizedMeasureArgs] = [
    {
        "shots": 1024,
        "wave": f"{wave_names}_{i}",
        "times": 100,
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_randomized.multiOutput(
    config_list,
    summoner_name="qurrent.randomized_measure",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurrent.randomized_measure.001", at location "qurrent.randomized_measure.001"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 079f5290-c0a2-490f-884e-993fd4b96329


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.001/qurryinfo.json...
| Exporting qurrent.randomized_measure.001/qurryinfo.json done.


'079f5290-c0a2-490f-884e-993fd4b96329'

In [6]:
experiment_randomized.multimanagers[multi_exps1]

<MultiManager(id="079f5290-c0a2-490f-884e-993fd4b96329",
  name="qurrent.randomized_measure.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-21 22:40:57',},
  exps_num=180)>

### d. Run post-processing at once


In [7]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

| "report.001" has been completed.


'079f5290-c0a2-490f-884e-993fd4b96329'

In [8]:
print("| Available results:")
for k, v in (
    experiment_randomized.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [9]:
experiment_randomized.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10")
][:2]

[{'purity': np.float64(1.161685791015625),
  'entropy': np.float64(-0.2162199060551491),
  'puritySD': np.float64(0.7768468260626491),
  'entropySD': np.float64(0.9647643727406497),
  'num_classical_registers': 10,
  'classical_registers': [0, 1],
  'classical_registers_actually': [0, 1],
  'all_system_source': 'independent',
  'purityAllSys': np.float64(2.219654083251953),
  'entropyAllSys': np.float64(-1.1503348606917023),
  'puritySDAllSys': np.float64(1.7811056406300567),
  'entropySDAllSys': np.float64(1.157654381565485),
  'num_classical_registers_all_sys': 10,
  'classical_registers_all_sys': None,
  'classical_registers_actually_all_sys': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  'errorRate': np.float64(-0.4902504205610562),
  'mitigatedPurity': np.float64(0.6605127781488721),
  'mitigatedEntropy': np.float64(0.5983416230538836),
  'counts_num': 100,
  'taking_time': 0.000428319,
  'taking_time_all_sys': 0.017495755,
  'counts_used': None,
  'input': {'num_qubits': 10,
   'selected_qub

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(
    experiment_randomized.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],   
    } if idx < 3 else False for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'079f5290-c0a2-490f-884e-993fd4b96329'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_randomized.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],  # selected qubits for the analysis
    } for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for 079f5290-c0a2-490f-884e-993fd4b96329


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurrent.randomized_measure.001/qurryinfo.json...
| Exporting qurrent.randomized_measure.001/qurryinfo.json done.


'079f5290-c0a2-490f-884e-993fd4b96329'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_randomized.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_randomized.multiRead(
    save_location=".",
    summoner_name="qurrent.randomized_measure.001",
)

| Retrieve qurrent.randomized_measure.001...
| at: qurrent.randomized_measure.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

In [16]:
expkeys_of_multi_exps1

['9e65fd4e-57f2-4c4b-b70d-ebf0029a8f5d',
 '082c0841-0129-4e82-89e0-30a237e32a66',
 '029aa268-c261-45c6-b7ff-e6e0de1c71af',
 '810302c7-712a-401a-9444-99aeb6789f56',
 'fc54a988-f481-4ae0-8004-f9c98b7aa776',
 '62993c88-41f8-4ec6-9d06-9c5ac5f0f57a',
 '07e1ecd9-a74b-413b-a112-7c313418944f',
 'b2c9f8bd-71bf-4547-8b71-f7febb13fc06',
 'bec68319-9270-41ff-8eb1-70fe2c8fad2e',
 'a4d97ecf-4e5e-4cb9-81d6-48484016235d',
 'd1656047-b74b-490b-9ccc-3bef8a64dca6',
 '113dd0e1-87d6-4ece-8700-47bddeb7b2d4',
 'a51e3ead-3eda-405a-92ef-cbe168d06481',
 'dec2b924-aede-44f6-822d-36da8ce2b720',
 'a1cb23c9-20d4-4114-8e17-632bdc483450',
 '1aba1dc0-3828-4579-bef0-26685d9adc43',
 '8093d284-202f-4ebe-980f-05c96a1af1ea',
 '5f53c4c6-58ea-40dd-a957-7e091a1244a0',
 '3e37b66b-3489-40e9-b716-f63257e7d36c',
 '2517669a-dbda-45bc-87e8-627ad4c652a2',
 'c87d1c0a-8d4e-4031-afb1-3860fa457d58',
 '623ccd57-1041-4870-bbbe-c284e5410d6d',
 '6af90b18-6f70-43f2-8dc7-4054b84b2993',
 '0b2808f7-7831-4bda-8ecf-4ad52088dc59',
 'a97ee9fe-478d-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET
AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev1
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  